# Del 2

1. I henhold til AAPCS32, så skal følgende heltallsregistere bevares av funksjonen:

   - `r4-r11` / `v1-v8` (delte verdier, osv.)
   - `r13` / `sp`
   
   Kalleren skal så bevare følgende heltallsregistere:

   - `r0-r3` / `a1-a4` (argumenter, returverdi)
   - `r12` / `ip`
   - `r14` / `lr`
   
   Hvis prosessoren støtter flyttallsregning, så skal

    - `s0-s15` / `d0-d7`/ `q0-q3` bevares av kalleren
    - `s16-s31` / `d8-d15` / `q4-q7` bevares av funksjonen

2. Funksjonen `fib` *skal* ta indekstallet i `r0` og bruke den til å regne ut resultatet. Dette *skal* returneres i `r0`. Argumentet er altså "brukt opp", og vi får resultatet i registeren som *for en stund siden* var argumentet.

3. For at dette programmet skal avslutte riktig, så måtte `main` lagre `lr` på starten, da den utfører en kall til `printf` -- altså at den ikke er en *bladfunksjon*. Et slik kall til en ekstern funksjon vil ellers "bruke opp" `lr`, sånn at programmet *ikke* klarer å gi tilbake kontrollen til operativsystemet.

# Del 3

1. Den største forskjellen, er antallet *direktiver* som brukes. I den håndskrevne versjonen, så er antall direktiver minimal: det erklæres en tekst-seksjon (kjørbar kode) og funksjonen `main` utpekes som *inngangspunkt* ("entry point"). I den genererte versjonen, så er instruksene og direktivene annerledes: rekkefølgen på trinnene er rokket om for størrelse og effektivitet, metadata lagres i direktivene, osv.
2. Det er ikke noe vesentlig forskjell mellom `-Os`, `-O2` og `-O3`, da selveste logikken er verken minne-intensiv eller gren-intensiv ("branch-heavy"). Forskjellene ligger hovedsakelig i direktivene, da størrelsen til programmet kan endre seg noe mellom variantene.
3. Diskusjonen dreier seg hovedsakelig om skillen mellom *reglement* ("policy") og *mekanisme* ("mechanism").

   - Det største argumentet *for* å bruke en kompilator framfor en assembler, er *portabilitet*: et kompilert språk formidler hovedsakelig *reglement*, mens assemblerspråk formidler hovedsakelig *mekanisme*: assemblerspråk er altså maskinspråk med forståelige navnelapper ("mnemonics").
   - På den andre siden, finnes det visse rutiner som *på lovlig vis* ikke kan implementeres i kompilerte språk (I/O, oppstartsrutiner, osv.) I slike tilfeller, er mekanisme den eneste fornuftige fortolkningen -- noe assembler er egnet for.

# Del 4
## Bakgrunn
Flyttall som følger standarden **IEEE 754** *skal* kunne representeres på formen
$$
    x = (-1)^{s}\cdot\left(1 + \frac{f}{2^M}\right)\cdot 2^{e - b}
$$
der $s$ er fortegnsbit-en, $f$ er *bitfelten* som inneholder brøkdelen, $M$ er lengden på bitfelten $f$, $e$ er eksponenten *med forskyvning* ("biased exponent") og *b* er forskyvningen. Dette kan formuleres på en lignende måte:
$$
    x = (-1)^{s} \cdot (1.f)_2 \cdot 2^{e-b}
$$
Da skal man kunne representere addisjon slik:
$$
    (1.a)_2 \cdot 2^{n} + (1.b)_2 \cdot 2^{m} = \left[(cd.e)_2 \cdot 2^{-c} \right] \cdot 2^{n + c}
$$
der $n,m: n \geq m$ er eksponentene, $a$, $b$ og $e$ er bitfelter med lengde $M$, og $c,d \in \{0, 1\}$.

Bitfeltet for $x$ er så gitt ved
$$
    X_{31:0} = s \cdot 2^{N-1} + e \cdot 2^{M} + f \equiv \mathtt{(s << (N-1)) | (e << M) | f} 
$$
For 32-bits flyttall, så er $M = 23$ og $b = 127$.

## Besvarelse

1. Vi har $2.0_{10} = (1.000\dots)_2 \cdot 2^{1} \equiv (1.000\dots)_2 \cdot 2^{128 - 127}$, sånn at $f = 0 \equiv \mathtt{0x00}$ og $e = 128 \equiv \mathtt{0x80}$. Bitfeltet for $2.0_{10}$ er så $X_{31:0} = \mathtt{(0x80 << 23) \, |\, 0x00} = \mathtt{0x40000000}$
2. Vi har $3.0_{10} = (1.100\dots)_2 \cdot 2^{1} \equiv (1.100\dots)_2 \cdot 2^{128 - 127}$, sånn at $f = \mathtt{1 << 22} \equiv \mathtt{0x400000}$ og $e = 128 \equiv \mathtt{0x80}$. Bitfeltet for $3.0_{10}$ er så $X_{31:0} = \mathtt{(0x80 << 23)\, |\, 0x400000} = \mathtt{0x40400000}$
3. Vi må først normalisere tallet: $0.50390625_{10} = 1.0078125_{10} \cdot 2^{-1} \equiv (1.0000001000\dots)_{2} \cdot 2^{-1}$